<!-- 
---

### Promemoria: questo 📘 `.NET Interactive` deve essere eseguito da VS Code con [questi prerequisiti](../PREREQS.md).

--- -->

# Ricetta II: 🍽️ Primo piatto
## 🧑‍🍳 Cuciniamo il nostro 🛢️🔥 primo piatto base

Questo quaderno è stato concepito per mostrare il modo più semplice per aggiungere l'intelligenza artificiale LLM a una vostra applicazione. 
È suddiviso in tre parti:

1. **Impostazione della chiave OpenAI o Azure OpenAI Service.** In questo modo è possibile utilizzare il notebook come una sorta di parco giochi. È sufficiente inserire la chiave una sola volta per iniziare: viene memorizzata localmente in un file chiamato "`settings.json`" sul disco. Assicuratevi che questo file non venga visualizzato pubblicamente da nessuna parte, come ad esempio sul vostro repo personale di GitHub.

2. **Ottenere un kernel 🔥 istanziato.** Con la chiave OpenAI o Azure OpenAI è possibile creare un kernel a cui inviare le istruzioni.
Quando si usa OpenAI, viene utilizzato per default il modello `text-davinci-003`; quando si usa Azure OpenAI c'è un'ulteriore impostazione dell'endpoint da considerare e inoltre viene chiesto esplicitamente il modello che si desidera utilizzare.

3. **Eseguire una semantic 🛢️ function.** Ok! Siete pronti a fare una richiesta in linguaggio naturale all'intelligenza artificiale LLM.

## Step 1️: Impostare la chiave OpenAI o Azure OpenAI Service

### 1.1: Il sistema vi chiederà di digitare la vostra chiave API segreta.

La variabile booleana sottostante `useAzureOpenAI` deve essere impostata su `false` se si utilizza una normale chiave OpenAI e su `true` se si dispone di una chiave Azure OpenAI Service. Qualunque sia l'impostazione della variabile, nella 👆 parte superiore 👆 della pagina apparirà un riquadro rettangolare che chiederà di inserire le credenziali.\
Queste credenziali sono memorizzate **localmente** sul proprio disco e non vanno da nessuna parte.

In [10]:
#!import ../config/Settings.cs

bool useAzureOpenAI = true;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

Error: Input request cancelled

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 62
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 44
   at Submission#16.Settings.AskAzureEndpoint(Boolean _useAzureOpenAI, String configFile)
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

RICORDA: è sufficiente eseguire questa operazione **una volta**. Il resto dei notebook utilizzerà la chiave API memorizzata sul vostro computer in questo importante passaggio.

### 1.2 Nel caso in cui si sia commesso un errore di immissione, seguire le seguenti istruzioni

#### 🟡 **SALTA QUESTO STEP** se la chiave è già stata impostata con successo

Premere il pulsante ▶️ solo se si desidera ripristinare quanto inserito in precedenza. Prima di farlo, è necessario "decommentare" due caratteri eliminando `//` all'inizio della riga 4.

In altre parole, cambiare `// Settings.Reset();` in `Settings.Reset();`. In questo modo si "decommenta" quella riga di codice, in modo che diventi attiva.

In [11]:
#!import ../config/Settings.cs

// Uncomment the line below to reset your settings and run step 1.1 again so it asks you for your API key
// Settings.Reset();

## Step 2️: Preparare un kernel 🔥 per cucinare il primo piatto

Congratulazioni! Siete a un terzo del percorso! Premete ▶️ per accedere alle credenziali memorizzate localmente, impostate nel primo passo. Questo passo carica il pacchetto Microsoft.SemanticKernel e prepara il resto del blocco note per far funzionare subito la funzione semantica.

In [12]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview

Configuration not found: ../config/settings.json

Please run the Setup Notebook (0-AI-settings.ipynb) to configure your AI backend first.



Error: System.Exception: Configuration not found, please setup the notebooks first using notebook 0-AI-settings.pynb
   at Submission#20.Settings.LoadFromFile(String configFile)
   at Submission#21.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Se si vede una linea di conferma in alto che inizia con `Installed Packages ... • Microsoft.SemanticKernel, ...` procedere al punto 3.

## Step 3️: Eseguire una funzione semantica 🛢️ in Semantic Kernel per 🔥cuocere 

### Step 3.1: Definire un prompt parametrizzato che accetta un singolo input

Il codice seguente è quello che chiamiamo "funzione semantica", che è quasi equivalente alla parola "prompt". 

In [13]:
string mySemanticFunctionInline = """
{{$input}}

Summarize the content above in less than 140 characters.
""";

Console.WriteLine("A string has been set to be used as a semantic function.");

A string has been set to be used as a semantic function.


Quindi la funzione semantica che stiamo per definire prende un testo `$input` e lo riassume in meno di 140 caratteri.

### Step 3.2: Configurare il prompt per renderlo più non deterministico (creativo) o deterministico (diretto)

#### 🔵 FAST TRACK Most people will just run the code below without much thought to tuning it, to start

L'impostazione `MaxTokens` determina il numero massimo di tokens da generare nel completamento. È il parametro più importante [setting](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions) da conoscere, perché ha un impatto su quanto spendete per ogni richiesta.

Inoltre, è possibile modellare in modo sottile l'uscita della risposta con gli altri due parametri. Per rendere la risposta più o meno "creativa", si può regolare l'impostazione `Temperatura' tra 0 e 1`. È inoltre possibile impostare il parametro `TopP` tra 0 (vocabolario più piccolo) e 1 (vocabolario più grande) per ottenere un risultato diverso.

In [14]:
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.SemanticFunctions;

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 1000, Temperature = 0.2, TopP = 0.5,
    }
};

var promptTemplate = new PromptTemplate(
    mySemanticFunctionInline, promptConfig, kernel
);

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

// "MySkill" is the name of the skill
// "Summary" is the name of the semantic function
var summaryFunction = kernel.RegisterSemanticFunction("MySkill", "Summary", functionConfig);

Console.WriteLine("A semantic function has been registered.");


Error: Microsoft.SemanticKernel.KernelException: ServiceNotFound: Text completion service not available
   at Microsoft.SemanticKernel.Configuration.KernelConfig.GetTextCompletionServiceIdOrDefault(String serviceId)
   at Microsoft.SemanticKernel.Kernel.GetService[T](String name)
   at Microsoft.SemanticKernel.Kernel.<CreateSemanticFunction>b__32_0()
   at Microsoft.SemanticKernel.Orchestration.SKFunction.SetAIBackend(Func`1 backendFactory)
   at Microsoft.SemanticKernel.Kernel.CreateSemanticFunction(String skillName, String functionName, SemanticFunctionConfig functionConfig)
   at Microsoft.SemanticKernel.Kernel.RegisterSemanticFunction(String skillName, String functionName, SemanticFunctionConfig functionConfig)
   at Submission#23.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exce

### Step 3.3: Set your input to the templated prompt and have the kernel 🔥 process it

In [15]:

var input = """
I think with some confidence I can say that 2023 is going to be the most exciting year that 
the AI community has ever had,” writes Kevin Scott, chief technology officer at Microsoft, 
in a Q&A on the company’s AI blog. He acknowledges that he also thought 2022 was the most 
exciting year for AI, but he believes that the pace of innovation is only increasing. 
This is particularly true with generative AI, which doesn’t simply analyze large data sets 
but is a tool people can use to create entirely new works. We can already see its promise 
in systems like GPT-3, which can do anything from helping copyedit and summarize text to 
providing inspiration, and DALL-E 2, which can create useful and arresting works of art 
based on text inputs. Here are some of Scott’s predictions about how AI will change the 
way we work and play.
""";
// Text source: https://www.microsoft.com/en-us/worklab/kevin-scott-on-5-ways-generative-ai-will-transform-work-in-2023

var summary = await kernel.RunAsync(input, summaryFunction);

Console.WriteLine(summary);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Microsoft.SemanticKernel.Kernel.RunAsync(ContextVariables variables, CancellationToken cancellationToken, ISKFunction[] pipeline)
   at Submission#24.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

🎉 Hai appena creato la tua prima funzione semantica usando Semantic Kernel. Congratulazioni 🔥!

> 🤔 **Ottieni il seguente `"Error: Throttling: Too many requests ..."` messaggio?** I servizi OpenAI risultano essere estremamente popolari in questi giorni. Se si utilizza la chiave per un account gratuito, questo messaggio apparirà spesso.

# ⏭️ I prossimi passi

Esempi più avanzati nei notebook disponibili nel repo GitHub all'indirizzo [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Vediamo le 🛢️ skills!](../e3-skills-rack/notebook.ipynb)